In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext
import time

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval_w_baseline

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()
print(list_city_dt)
print(len(list_city_dt))

['berlin_db=0.50', 'berlin_db=1.00', 'berlin_db=2.00', 'berlin_db=4.00', 'frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00', 'lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00', 'paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']
16


In [3]:
#Subset to berlin

list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find('berlin') > -1):
        list_city_dt_subset.append(city_db)

list_city_dt_subset.remove('berlin_db=0.50')
print(list_city_dt_subset)


['berlin_db=1.00', 'berlin_db=2.00', 'berlin_db=4.00']


In [5]:


RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

for base_file in list_city_dt_subset:
# for base_file in ['lyon_db=0.50']:

    list_results = []
    dict_out = {'instance': [],
        'city': [],
        'demand_baseline': [],
        'type': [],
        'outsourcing_cost_multiplier': [],
        'region_multiplier': [],
        'global_multiplier': [],
        'model': [],
        'max_n_shifts': [],
        'workforce_size': [],
        'wage_costs': [],
        'objective_value': [],
        'objective_value_post_wage': []}

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    expand_workforce_to_regions=True
    workforce_dict = {}

    #create dictionary for evaluating results by city/demand baseline
    results_out = f"../results/{base_file}.json"

    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    # for OC in [1.2]:
        for model in ['fixed', 'flex', 'partflex']:
        # for model in ['fixed']:
            if model == 'partflex':
                for max_n_shifts in range(2,5):
                    start_time = time.time()
                    dict_baseline, dict_results = run_roster_solver_objval_w_baseline(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts, expand_workforce_to_regions=True)
                    end_time = time.time()
                    run_time = (end_time - start_time)
                    dict_baseline['type'] = ['baseline']
                    dict_results['type'] = ['model']
                    df_ = pd.DataFrame(dict_baseline)
                    list_results.append(df_)
                    # list_results_all.append(df_)
                    df_ = pd.DataFrame(dict_results)
                    list_results.append(df_)
                    # list_results_all.append(df_)
                    for key in dict_out.keys():
                        dict_out[key].extend(dict_baseline[key])
                        dict_out[key].extend(dict_baseline[key])
                    print(f'Base: {base_file}, OC: {OC}, model: {model}-{max_n_shifts}, run_time: {run_time}')
            else:
                start_time = time.time()
                dict_baseline, dict_results = run_roster_solver_objval_w_baseline(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, expand_workforce_to_regions=True)
                end_time = time.time()
                run_time = (end_time - start_time)
                dict_baseline['type'] = ['baseline']
                dict_results['type'] = ['model']
                df_ = pd.DataFrame(dict_baseline)
                list_results.append(df_)
                # list_results_all.append(df_)
                df_ = pd.DataFrame(dict_results)
                list_results.append(df_)
                # list_results_all.append(df_)
                for key in dict_out.keys():
                    dict_out[key].extend(dict_baseline[key])
                    dict_out[key].extend(dict_baseline[key])
                print(f'Base: {base_file}, OC: {OC}, model: {model}, run_time: {run_time}')
    #export optimal workforce size values
    with open(results_out, 'w') as f:
        json.dump(dict_out, f, indent=2)
    df_results = pd.concat(list_results, ignore_index = True)
    df_results.to_excel(f'../results/{base_file}.xlsx', index = False)



Base: berlin_db=1.00, OC: 1.2, model: fixed, run_time: 28.06146216392517
Base: berlin_db=1.00, OC: 1.2, model: flex, run_time: 338.5765542984009
Base: berlin_db=1.00, OC: 1.2, model: partflex-2, run_time: 28.1957106590271
Base: berlin_db=1.00, OC: 1.2, model: partflex-3, run_time: 302.87793469429016
Base: berlin_db=1.00, OC: 1.2, model: partflex-4, run_time: 338.15548062324524
Base: berlin_db=1.00, OC: 1.5, model: fixed, run_time: 31.991613149642944
Base: berlin_db=1.00, OC: 1.5, model: flex, run_time: 2221.0661425590515
Base: berlin_db=1.00, OC: 1.5, model: partflex-2, run_time: 33.103434324264526
Base: berlin_db=1.00, OC: 1.5, model: partflex-3, run_time: 279.9790503978729
Base: berlin_db=1.00, OC: 1.5, model: partflex-4, run_time: 2858.4214911460876
Base: berlin_db=1.00, OC: 1.8, model: fixed, run_time: 34.17139792442322
Base: berlin_db=1.00, OC: 1.8, model: flex, run_time: 310.0044574737549
Base: berlin_db=1.00, OC: 1.8, model: partflex-2, run_time: 31.966629028320312
Base: berlin_

In [ ]:
# list_results_all = []
# df_results_all = pd.concat(list_results_all, ignore_index = True)
# df_results_all.to_excel(f'../results/all_cities_and_demand_baselines.xlsx', index = False)
